# Preparação do ambiente

## Dados

In [1]:
!wget -P "./EEG-data" https://physionet.org/static/published-projects/eegmat/eeg-during-mental-arithmetic-tasks-1.0.0.zip

--2022-10-23 21:30:37--  https://physionet.org/static/published-projects/eegmat/eeg-during-mental-arithmetic-tasks-1.0.0.zip
Resolving physionet.org (physionet.org)... 18.18.42.54
Connecting to physionet.org (physionet.org)|18.18.42.54|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 183634285 (175M) [application/zip]
Saving to: ‘./EEG-data/eeg-during-mental-arithmetic-tasks-1.0.0.zip’

eeg-during-mental-a 100%[===================>] 175.13M  12.0MB/s    in 14s     

2022-10-23 21:30:52 (12.3 MB/s) - ‘./EEG-data/eeg-during-mental-arithmetic-tasks-1.0.0.zip’ saved [183634285/183634285]



## Bibliotecas

In [2]:
!pip install pyEDFlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.4 MB 5.0 MB/s 


In [3]:
import pandas as pd
import pyedflib
import zipfile
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import scipy as sp

from scipy import signal
from scipy.signal import hilbert

In [4]:
from numpy.random import seed
seed(42)

## Funções

In [5]:
def filtragem(signal, ord=5,f_pa=.05, f_pb=50, Fs=500):
  #Inputs
  #- ord: Ordem do filtro Butterworth
  #- f_pa: Frequencia passa-alta do filtro
  #- f_pb: Frequência passa-baixa do filtro
  #- Fs: Frequência de amostragem do sinal
  #- signal: Sinal a ser filtrado
  #Output
  #- signal_f: Sinal filtrado

  f_pb_n = f_pb/Fs
  f_pa_n = f_pa/Fs
  signal_f = np.zeros(signal.shape)

  for col in range(signal.shape[1]):
      sos = sp.signal.butter(ord,[f_pa_n,f_pb_n],btype='band',analog=False,output='sos')
      signal_f[:,col] = sp.signal.sosfiltfilt(sos,signal[:,col])
  return signal_f

In [47]:
def Features(sinal,fs=500, modo_c = 0):
  #Inputs
  #- sinal: sinal de onde serão tiradas as características
  #- fs: frequência de amostragem do sinal
  #- modo_c: se todas as características estarão numa única coluna

  #Output
  #- sig_features: 8 características do sinal: frequência instântanea no tempo;
  #                potência das ondas alfa, beta, gamma, delta, theta; média e
  #                desvio-padrão.

  n_col = sinal.shape[1]


  # sig_features = np.ones((int(1024/n_col),n_col))
  sig_features = np.ones((8,n_col))

  # A extração de características é feita canal por canal
  for i in range(sinal.shape[1]):

    sinal_c = sinal[:,i]

    # Encontra a frequência instantanea do canal utilizando a transformada de Hilbert
    analytic_signal = hilbert(sinal_c)
    instantaneous_phase = np.unwrap(np.angle(analytic_signal))
    instantaneous_frequency = np.abs((np.diff(instantaneous_phase) / (2.0*np.pi) * fs))
    instantaneous_frequency = np.mean(instantaneous_frequency).reshape((-1,1))

    # Determina a densidade do espectro de potência do canal
    f, psd = signal.welch(sinal_c, 500)

    alfa = np.trapz(y = psd[4:6], x = f[4:6]).reshape((-1,1))
    beta = np.trapz(y = psd[6:15], x = f[6:15]).reshape((-1,1))
    gamma = np.trapz(y = psd[15:50], x = f[15:50]).reshape((-1,1))
    delta = np.trapz(y = psd[0:2], x = f[0:2]).reshape((-1,1))
    theta = np.trapz(y = psd[2:4], x = f[2:4]).reshape((-1,1))

    sig_mean = np.mean(sinal_c).reshape((-1,1))

    sig_std = np.std(sinal_c).reshape((-1,1))

    # feat = np.concatenate((instantaneous_frequency,delta,theta,alfa,beta,gamma,sig_mean,sig_std)).reshape((-1,1))

    sig_features[0,i] = instantaneous_frequency
    sig_features[1,i] = delta
    sig_features[2,i] = theta
    sig_features[3,i] = alfa
    sig_features[4,i] = beta
    sig_features[5,i] = gamma
    sig_features[6,i] = sig_mean
    sig_features[7,i] = sig_std

    if n_col == 1:
      sig_features[i*256:(i+1)*256] = feat
  
  return np.transpose(sig_features)

In [7]:
def segmenta(sinal, passo, dimensoes):
  #Inputs
  #- sinal: sinal a ser segmentado
  #- passo: quantas amostras irá avançar de um segmento para outro
  #- dimensoes: Dimensões esperada do output

  #Output
  #- segmentado: Sinal segmentado em segmentos de 2*passo de duração e com
  # overlap de 1 passo entre eles

  segmentado = np.zeros(dimensoes)
  cont = 0
  for linha in range(sinal.shape[0]):
      sinal_s = sinal[linha,:,:]
      for i in range(1000000):
          if ~np.any(sinal_s[i*passo:(i+2)*passo,:]) or sinal_s[i*passo:(i+2)*passo,:].shape[0] != int(2*passo):
              break
          segmentado[cont,:,:] = sinal_s[i*passo:(i+2)*passo,:]
          cont = cont+1
  return segmentado

# Gerando o dataset

In [8]:
with zipfile.ZipFile("./EEG-data/eeg-during-mental-arithmetic-tasks-1.0.0.zip") as z:
    lista_arquivos = z.namelist()
  
lista_arquivos.remove('eeg-during-mental-arithmetic-tasks-1.0.0/subject-info.csv')
lista_arquivos.remove('eeg-during-mental-arithmetic-tasks-1.0.0/SHA256SUMS.txt')
lista_arquivos.remove('eeg-during-mental-arithmetic-tasks-1.0.0/README.txt')
lista_arquivos.remove('eeg-during-mental-arithmetic-tasks-1.0.0/RECORDS')

In [9]:
repouso = []
ativos = []

for i in lista_arquivos:
    if(i.endswith('_2.edf')):
        ativos.append(i)
    else:
        repouso.append(i)

with zipfile.ZipFile("./EEG-data/eeg-during-mental-arithmetic-tasks-1.0.0.zip") as z:
    arq = pyedflib.EdfReader(z.extract(repouso[0]))
    labels = arq.getSignalLabels()
    arq.close()
labels.pop()

##Coletando os dados de antes as perguntas
dfs_repouso = [None] * 36
with zipfile.ZipFile("./EEG-data/eeg-during-mental-arithmetic-tasks-1.0.0.zip") as z:
    for i in repouso:
        arq = pyedflib.EdfReader(z.extract(i))
        j = int(i[-8]+i[-7])
        x = pd.DataFrame()
        for k in range(0,len(labels)):
            x[labels[k]] = arq.readSignal(k)
        dfs_repouso[j] = x
        arq.close()

##Coletando os dados de durante as perguntas
dfs_ativos = [None] * 36
with zipfile.ZipFile("./EEG-data/eeg-during-mental-arithmetic-tasks-1.0.0.zip") as z:
    for i in ativos:
        arq = pyedflib.EdfReader(z.extract(i))
        j = int(i[-8]+i[-7])
        x = pd.DataFrame()
        for k in range(0,len(labels)):
            x[labels[k]] = arq.readSignal(k)
        dfs_ativos[j] = x    
        arq.close()

In [10]:
mult = int(5e3) #Nº de amostras na intersecção
seq_rest = 0
seq_active = 0

# Nome de cada coluna do dataframe (para referência)
# nomes = ['Fp1','Fp2','F3','F4','F7','F8','T3','T4','C3','C4','T5','T6','P3','P4','O1','O2','Fz','Cz','Pz','A2-A1','ECG','State']
nomes = ['Fp1','Fp2','F3','F4','F7','F8','T3','T4','C3','C4','T5','T6','P3','P4','O1','O2','Fz','Cz','Pz','A2-A1']

# A quantidade de cada tipo de segmento definida com base no artigo
rest_seqs = np.zeros((601,10000,len(nomes)))
acti_seqs = np.ones((180,10000,len(nomes)))

for ativo in dfs_ativos:
  aux = ativo.to_numpy()
  for i in range(1000000):
    # Quando a segmentação resultar em um segmento com menos de 10k de amostras ou resulte em
    # um vetor vazio (o indice de início é maior que o vetor), acaba a segmentação deste arquivo
    # e vai para o próximo.
    if ~np.any(aux[i*mult:(i+2)*mult,:]) or aux[i*mult:(i+2)*mult,:].shape[0] != 10000:
      break
    acti_seqs[seq_active,:,:] = filtragem(aux[i*mult:(i+2)*mult,:])
    seq_active = seq_active+1

for repouso in dfs_repouso:
  aux = repouso.to_numpy()
  for i in range(1000000):
    # Quando a segmentação resultar em um segmento com menos de 10k de amostras ou resulte em
    # um vetor vazio (o indice de início é maior que o vetor), acaba a segmentação deste arquivo
    # e vai para o próximo.
    if ~np.any(aux[i*mult:(i+2)*mult,:]) or aux[i*mult:(i+2)*mult,:].shape[0] != 10000:
      break
    rest_seqs[seq_rest,:,:] = filtragem(aux[i*mult:(i+2)*mult,:])
    seq_rest = seq_rest+1

# rest_seqs = np.concatenate((rest_seqs,np.zeros((601,10000,1))),axis=2)
# acti_seqs = np.concatenate((acti_seqs,np.ones((180,10000,1))),axis=2)

In [11]:
rest_train = list(np.random.choice(601,480,replace=False))
rest_test = [x for x in range(601) if x not in rest_train]

active_train = list(np.random.choice(180,144,replace=False))
active_test = [x for x in range(180) if x not in active_train]

rest_train.sort()
active_train.sort()
rest_test.sort()
active_test.sort()

print(rest_train[:5])
print(active_train[:5])
print(rest_test[:5])
print(active_test[:5])

[0, 2, 3, 4, 5]
[0, 1, 2, 3, 5]
[1, 13, 14, 20, 21]
[4, 6, 8, 18, 26]


In [12]:
rest_train = rest_seqs[rest_train,:,:]
active_train = acti_seqs[active_train,:,:]
rest_test = rest_seqs[rest_test,:,:]
active_test = acti_seqs[active_test,:,:]

rest_train = rest_train[:,:,[0,4,13,18]]
active_train = active_train[:,:,[0,4,13,18]]
rest_test = rest_test[:,:,[0,4,13,18]]
active_test = active_test[:,:,[0,4,13,18]]

In [13]:
rest_train = segmenta(rest_train,125,(37920,250,rest_train.shape[2]))
active_train = segmenta(active_train,125,(11376,250,active_train.shape[2]))
rest_test = segmenta(rest_test,125,(9559,250,rest_test.shape[2]))
active_test = segmenta(active_test,125,(2844,250,active_test.shape[2]))

In [14]:
print(rest_train.shape)
print(active_train.shape)
print(rest_test.shape)
print(active_test.shape)

(37920, 250, 4)
(11376, 250, 4)
(9559, 250, 4)
(2844, 250, 4)


In [52]:
# x = np.zeros((rest_train.shape[0],256,20))
# x = np.zeros((rest_train.shape[0],1024))
x = np.zeros((rest_train.shape[0],4,8))
for segm in range(rest_train.shape[0]):
    x[segm,:] = Features(rest_train[segm,:,:])
rest_train = np.concatenate((x, np.zeros((x.shape[0],1))),axis=1)

# x = np.zeros((active_train.shape[0],256,20))
# x = np.zeros((active_train.shape[0],1024))
x = np.zeros((active_train.shape[0],4,8))
for segm in range(active_train.shape[0]):
    x[segm,:] = Features(active_train[segm,:,:])
active_train = np.concatenate((x, np.ones((x.shape[0],1))),axis=1)

ValueError: ignored

In [ ]:
np.concatenate((x, np.zeros((x.shape[0],1))),axis=1)

In [ ]:
# x = np.zeros((rest_test.shape[0],256,4))
# x = np.zeros((rest_test.shape[0],1024))
x = np.zeros((rest_test.shape[0],4,8))
for segm in range(rest_test.shape[0]):
    x[segm,:] = Features(rest_test[segm,:,:])
rest_test = np.concatenate((x, np.zeros((x.shape[0],1))),axis=1)

# x = np.zeros((active_test.shape[0],256,4))
# x = np.zeros((active_test.shape[0],1024))
x = np.zeros((active_test.shape[0],4,8))
for segm in range(active_test.shape[0]):
    x[segm,:] = Features(active_test[segm,:,:])
active_test = np.concatenate((x, np.ones((x.shape[0],1))),axis=1)

In [ ]:
train = np.concatenate((rest_train,active_train),axis=0)
test = np.concatenate((rest_test,active_test),axis=0)

In [ ]:
train_data, y_train = train[:,:-1], train[:,-1]
test_data, y_test = test[:,:-1], test[:,-1]

In [ ]:
# y_train = np.mean(y_train, axis=1,dtype=np.int16).reshape((-1,1))
# y_test = np.mean(y_test, axis=1,dtype=np.int16).reshape((-1,1))

y_train = y_train.reshape((-1,1))
y_test = y_test.reshape((-1,1))

# Salvar o dataset

In [ ]:
np.savez_compressed('Desespero_train',data = train_data, labels = y_train)

In [ ]:
np.savez_compressed('Desespero_test',data = test_data, labels = y_test)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!cp '/content/Train1C.npz' '/content/drive/MyDrive'
!cp '/content/Test1C.npz' '/content/drive/MyDrive'